# Lecture 3.2: FRED

In [73]:
import pandas as pd
import requests
from urllib.parse import urlencode
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import plotly.express as px
import plotly.graph_objects as go




In [74]:
try:
    from fredapi import Fred
except ImportError:
    !pip install fredapi
    from fredapi import Fred

## Macroeconomic Data and Fred API

**What is an API?**  
In contrast to a user interface, which connects a computer to a person, an application programming interface (API) connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them. 

In basic terms, APIs just allow applications to communicate with one another. For the APIs we are concerned right now--web based APIs that return data in response to a request made by us--**they allow us to get data from outside sources by sending an API a request detailing the information we want. Then the API will "respond" with the requested data to us.**

<img src="images/what_is_an_API.png" alt="What is an API" style="width:600px;height:200px;">
<center>Source: Perry Eising, "What exactly is an API?"</center>

### General Way of Interacting with an API

First, we will use the most common way to access the data through an API without using any packages other may have already built for a particular site. Usually this is the way you would interact with an API.  

#### Step 1: Get the API Key
**In most cases, you will need to get an API key in order to access an API.** For many resources, it involves some paperwork to apply and/or limited free usage, so it is good practice to keep your API keys private as long as it is possible. In this lecture notebook, we will use macroeconomic data from FRED, which is one of the most famous and convenient sources of economic data. For FRED, the process of getting an API key is simple. Request the API key [here](https://fred.stlouisfed.org/docs/api/api_key.html). 

In [75]:
# For demo purposes, we are leaving our API key here. You should always try to avoid this. 
api_key = "dab081fe5e028d7fc65114e0c7f2cf6b"

#### Step 2: Learn to use the API
Using an API is like ordering food at a restaurant with a menu. To have a delicious meal, we have to know what food the restaurant offers, and any other additional information (for example, how would you like your steak). Similarly, it is very important for us to know what requests an API take through the API documentation. **The API documentation will inform us about how we can use specific instructions to obtain the data that we want, and what the returned data would look like.** Look up the Fred API's documentation [here](https://fred.stlouisfed.org/docs/api/fred/series_observations.html). 


<div class="alert alert-info">
<b> Example: "api.stlouisfed.org/fred/series/observations?series_id=GNPCA&api_key=abcdefghijklmnopqrstuvwxyz123456" <br>  
  
Endpoint: "api.stlouisfed.org/fred/series/observations"  <br>
Parameters: series_id=GNPCA, api_key=abcdefghijklmnopqrstuvwxyz123456
</div>

## Try the following link to see what the API will return!  

 https://api.stlouisfed.org/fred/series/observations?series_id=GNPCA&api_key=dab081fe5e028d7fc65114e0c7f2cf6b&file_type=json

In [76]:
# I want to make a fetch command for the URL in the previous cell
response = requests.get("https://api.stlouisfed.org/fred/series/observations?series_id=GNPCA&api_key=dab081fe5e028d7fc65114e0c7f2cf6b&file_type=json")

In [ ]:
data=response.json()
print(data)


In [ ]:
GNPCA = pd.DataFrame(data['observations'])
GNPCA.head()

In [ ]:
GNPCA = GNPCA[["date", "value"]]
GNPCA.rename(columns={"value": "GNPCA"}, inplace=True)
GNPCA.head()

In [80]:
GNPCA.to_csv('GNPCA.csv')

## Switch to a quarterly series 

GDPC1

https://fred.stlouisfed.org/series/GDPC1



In [ ]:
response2 = requests.get("https://api.stlouisfed.org/fred/series/observations?series_id=GDPC1&api_key=dab081fe5e028d7fc65114e0c7f2cf6b&file_type=json")
GDPC1 = pd.DataFrame(response2.json()['observations'])
GDPC1


In [ ]:
# drop the unwanted columns
GDPC1 = GDPC1[["date", "value"]]
# rename the columns using .loc
GDPC1 = GDPC1.assign(GDPC1=GDPC1['value'])
GDPC1 = GDPC1.drop(columns='value')
GDPC1

###  Lets try to get the data for the recessions

Series name is USRECD


In [83]:
response = requests.get("https://api.stlouisfed.org/fred/series/observations?series_id=USRECD&api_key=dab081fe5e028d7fc65114e0c7f2cf6b&file_type=json")

In [ ]:
data=response.json()
USRECD = pd.DataFrame(data['observations'])
USRECD.head()

In [ ]:
USRECD = USRECD[["date", "value"]]
USRECD.rename(columns={"value": "USRECD"}, inplace=True)
USRECD.head()

### Let's Merge GDP and Recession Bars

In [ ]:
# Merge the two dataframes
GNPRED_df = pd.merge(GDPC1, USRECD, on="date")
GNPRED_df

In [ ]:
GNPRED_df['date'] = pd.to_datetime(GNPRED_df['date'])
GNPRED_df.set_index('date', inplace=True)
GNPRED_df


In [ ]:
#Let's just keep data after 1945
GNPRED_df = GNPRED_df[GNPRED_df.index > '1945-01-01']
GNPRED_df

In [ ]:
GNPRED_df.dtypes

In [107]:

GNPRED_df.loc[:, 'USRECD'] = pd.to_numeric(GNPRED_df['USRECD'], errors='coerce')
GNPRED_df.loc[:, 'GDPC1'] = pd.to_numeric(GNPRED_df['GDPC1'], errors='coerce')

###  Lets see if we can plot it together?

In [ ]:

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(GNPRED_df.index, GNPRED_df['GDPC1'], label='GDPC1', color='blue')

ax.fill_between(GNPRED_df.index, 0, 1, where=GNPRED_df['USRECD']==1, 
                color='grey', alpha=0.3, transform=ax.get_xaxis_transform(), label='US Recession')

ax.set_xlabel('Date')
ax.set_ylabel('GReal Gross National Product')
ax.set_title('Real Gross National Product and US Recession Periods')
ax.legend()

plt.show()


In [ ]:

x_values = GNPRED_df.index
line = go.Scatter(x=x_values, y=GNPRED_df['GDPC1'], mode='lines', name='GDPC1')

bar = go.Bar(x=x_values, y=GNPRED_df['USRECD']*max(GNPRED_df['GDPC1']), name='US Recession', marker_color='grey', opacity=0.5)

fig = go.Figure(data=[line, bar])
fig.update_layout(title='Real Gross National Product and US Recession Periods',
                  xaxis_title='Date',
                  yaxis_title='Real Gross National Product',
                  barmode='overlay',
                  bargap=0.0001, 
                  )

fig.show()


### Site-Specific Prebuilt Packages

Now we will switch gears to use some prebuilt packages to access FRED API. Notice that the prebuilt packages are not necessarily available for every API. But here we will use the "FredAPI" package developed by Mortada Mehyar. Documentation [here](https://github.com/mortada/fredapi). 

In [ ]:

fred = Fred(api_key=api_key)
GDP_fredapi = pd.DataFrame(fred.get_series('GDPCA'))
GDP_fredapi.head(10)

In [124]:
# Let's get the data for the US Recession
USRECD_fredapi = pd.DataFrame(fred.get_series('USRECD'))

In [ ]:
# Merge the two dataframes
GDP_USRECD_df = pd.merge(GDP_fredapi, USRECD_fredapi, left_index=True, right_index=True)
GDP_USRECD_df


In [ ]:
#Lets rename the columns
GDP_USRECD_df.columns = ['GDP', 'USRECD']
GDP_USRECD_df.head()

In [ ]:
#plot in plotly
fig = px.line(GDP_USRECD_df, x=GDP_USRECD_df.index, y='GDP', title='US GDP and Recession Periods')
fig.add_bar(x=GDP_USRECD_df.index, y=GDP_USRECD_df['USRECD']*max(GDP_USRECD_df['GDP']), name='US Recession', opacity=0.5)
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='GDP')
fig.show()

In [ ]:
# Lets redo plot after 1945   
PW_GDP_USRECD_df = GDP_USRECD_df[GDP_USRECD_df.index > '1945-01-01']
PW_GDP_USRECD_df

In [ ]:
#potly with     make recessson bars grey
fig = px.line(PW_GDP_USRECD_df, x=PW_GDP_USRECD_df.index, y='GDP', title='US GDP and Recession Periods')
fig.add_bar(x=PW_GDP_USRECD_df.index, y=PW_GDP_USRECD_df['USRECD']*max(PW_GDP_USRECD_df['GDP']), name='US Recession', opacity=0.5, marker_color='grey')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='GDP')
fig.show()

In [ ]:
# can we inspect the data for 2008 and 2009
PW_GDP_USRECD_df[(PW_GDP_USRECD_df.index > '2006-01-01') ]